# LAB: Multiple Correspondence Analysis sobre datos de opinión pública

## Introducción

En este caso, trabajaremos con datos de una encuesta de opinión pública realizada en la CABA en el año 2007. Uno de los objetivos de la encuesta era encontrar predictores sobre el autoposicionamiento ideológico de la población. Para ello se realizaron una serie de afirmaciones (vinculadas al rol del estado, a ciertos derechos y al tipo de políticas necesarias para combatir la inseguridad) y se solicitó a los entrevistados que respondieran su nivel de acuerdo con dichas afirmaciones.

A los efectos de este LAB hemos recodificado las respuestas en tres grupos: 
    
   - Acuerdo (Muy o bastante de acuerdo)
   - Desacuerdo (muy o bastante en desacuerdo)
   - No sabe
   
**Objetivo:** Tenemos dos objetivos:

1. representar el dataset en un espacio bidimensional formado por las X afirmaciones relevadas
2. encontrar patrones en las respuestas de los encuestados


In [ ]:
import mca
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("./Data/eti285.csv")
df2 = pd.get_dummies(df.iloc[:,range(33,df.shape[1])])

In [ ]:
ncols = 10
mca_ben = mca.MCA(df2, ncols=ncols)

Observermos la inercia y la proporción de la varianza explicada.

In [ ]:
mca_ben.L

Podemos ver que con dos factores logramos explicar alrededor del 78% de la varianza.

In [ ]:
mca_ben.expl_var(greenacre=False, N=2), mca_ben.expl_var(greenacre=False, N=2).sum()

### Análisis gráfico

Recuperamos las funciones para plotear las filas y columnas en el espacio bidimensional.

In [ ]:
def plot_factors_ind_mca(mca_object, X, annotate = True):
    
    ''' Plot individuals in two first factors of MCA Analysis Arguments:
        #################################################
        
        mca_objet: object with MCA model
        X: DataFrame with data
    '''
    points = mca_object.fs_r(N=2).T
    labels = list(X.index)
    
    fig, ax = plt.subplots()
    plt.axhline(0, color='gray')
    plt.axvline(0, color='gray')
    ax.scatter(points[0], points[1])
   
    if annotate == True:
        for label, x, y in zip(labels, *points):
            plt.annotate(label, xy=(x, y))
        plt.xlabel('Factor 1')
        plt.ylabel('Factor 2')
    
    else:
        plt.xlabel('Factor 1')
        plt.ylabel('Factor 2')


def plot_factors_vars_mca(mca_object, X):
    
    ''' Plot individuals in two first factors of MCA Analysis Arguments:
        #################################################
        
        mca_objet: object with MCA model
        X: DataFrame with data
    '''
    points = mca_object.fs_c(N=2).T
    
    noise = 0.09 * (np.random.rand(*points.shape) - 0.6)
    points = noise + points
    labels = list(X.columns)
    
    fig, ax = plt.subplots()
    plt.axhline(0, color='gray')
    plt.axvline(0, color='gray')
    ax.scatter(points[0], points[1])
   
    for label, x, y in zip(labels, *points):
        plt.annotate(label, xy=(x, y))
    
    plt.xlabel('Factor 1')
    plt.ylabel('Factor 2')

Observemos ahora como se agrupan los casos en el espacio bidimensional

In [ ]:
plot_factors_ind_mca(mca_ben,df2,False)

¿Qué representa el espacio bidimensional? Para avanzar en la interpretación, podemos plotear las variables dummy generadas.

In [ ]:
plot_factors_vars_mca(mca_ben, df2)

El cuadrante 2 pareciera agrupar las respuestas más vinculadas a posiciones progresistas:
    
   - Acuerdo con intervención estatal
   - Desacuerdo con incremento de penas
   - Desacuerdo con arancelamiento de universidades

El tercer cuadrante pareciera agrupar a los más indecisos.

El cuadrante cuatro, las posiciones más conservadoras:
    
   - Acuerdo con incremento de penas
   - Desacuerdo con propiedad estatal del empresas de servicios
   - Acuerdo con arancelamiento de universidades, etc.

### Capacidad de clasificación de los componentes utilizados

Veamos ahora en qué medida la reducción de la dimensionalidad efectuada permite realizar una clasificación adecuada del autoposicionamiento ideológico de los entrevistados.

In [ ]:
fs_c_sup = mca_ben.fs_c_sup(pd.get_dummies(df['R_autoposicionamiento_ideologico']), N=2)
points = np.vstack((mca_ben.fs_c(N=2),fs_c_sup)).T

noise = 0.09 * (np.random.rand(*points.shape) - 0.6)
points = noise + points
labels = list(df2.columns.append(pd.get_dummies(df['R_autoposicionamiento_ideologico']).columns))

fig, ax = plt.subplots()
plt.axhline(0, color='gray')
plt.axvline(0, color='gray')
ax.scatter(points[0], points[1])
    
for label, x in zip(labels, points.T):
    plt.annotate(label, (x[0], x[1]))

plt.xlabel('Factor 1')
plt.ylabel('Factor 2')